In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
os.getcwd()

'd:\\workspace\\git_projects\\Medicare-Claim-Fraud-Detection\\scripts\\validation'

In [4]:
# beneficiary data
# bene_train = pd.read_csv('../data/Train_Beneficiarydata-1542865627584.csv')
bene_test= pd.read_csv('../../data/Test_Beneficiarydata-1542969243754.csv')

# inpatient data
inp_test = pd.read_csv('../../data/Test_Inpatientdata-1542969243754.csv')
# inp_train = pd.read_csv('../data/Train_Inpatientdata-1542865627584.csv')

# outpatient data
outp_test= pd.read_csv('../../data/Test_Outpatientdata-1542969243754.csv')
# outp_train = pd.read_csv('../data/Train_Outpatientdata-1542865627584.csv')

#label data
# label_train = pd.read_csv('../data/Train-1542865627584.csv')
label_test = pd.read_csv('../../data/Test-1542969243754.csv')

### beneficiary data

In [4]:
# bene_train.head()

In [6]:
bene_test.shape

(63968, 25)

In [7]:
bene_test.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')

In [7]:
bene_test['RenalDiseaseIndicator'].value_counts()

RenalDiseaseIndicator
0    118978
Y     19578
Name: count, dtype: int64

In [8]:
# this value indicates whether the beneficiary has renal disease
bene_test['RenalDiseaseIndicator'] = bene_test['RenalDiseaseIndicator'].replace('Y',1).astype(int)
bene_test['RenalDiseaseIndicator'].value_counts()

RenalDiseaseIndicator
0    53408
1    10560
Name: count, dtype: int64

In [9]:
# Data Reduction: Delete DOD DOB and change it into new feature 'Age' and 'Alive'
for col in ['DOB','DOD']:  
    bene_test[col] = pd.to_datetime(bene_test[col])
    

In [10]:
bene_test['BirthYear'] = bene_test['DOB'].dt.year
# Greatest Date of Death in the TRAIN set for beneficiaries
max_bene_DOD = max(bene_test['DOD'].unique()[1:])
max_bene_DOD

Timestamp('2009-12-01 00:00:00')

In [11]:
# For all NAN DODs filling the greatest Date of Death
bene_test['DOD'].fillna(value=max_bene_DOD, inplace=True)

# Calculating Age from DOD and DOB
bene_test['Age'] = round(((bene_test['DOD'] - bene_test['DOB']).dt.days)/365,0)


In [12]:
bene_test['Age'].describe()

count    63968.000000
mean        73.700553
std         12.809561
min         26.000000
25%         68.000000
50%         75.000000
75%         82.000000
max        101.000000
Name: Age, dtype: float64

In [13]:
bene_test['Alive'] = bene_test['DOD'].apply(lambda x: 1 if x != x else 0)
bene_test['Alive'].value_counts()

Alive
0    63968
Name: count, dtype: int64

In [14]:
# NoOfMonths_PartACOV and NoOfMonths_PartBCOV¶
bene_test.groupby('NoOfMonths_PartACov').count()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive
NoOfMonths_PartACov,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405,405
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
6,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
8,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
9,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [15]:
# NoOfMonths_PartBCOV¶
bene_train.groupby('NoOfMonths_PartBCov').count()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive
NoOfMonths_PartBCov,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675
1,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
2,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
3,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
4,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35
5,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
6,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282
7,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63
8,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71


In [15]:
bene_test.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'BirthYear', 'Age', 'Alive'],
      dtype='object')

In [16]:
diseases = ['ChronicCond_Alzheimer','ChronicCond_Heartfailure',
           'ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary',
          'ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis',
           'ChronicCond_rheumatoidarthritis','ChronicCond_stroke']

In [17]:
# converting diseases columns to binary column as each disease say 1: Yes, 2: No

for disease in diseases:
    bene_test[disease] = bene_test[disease].replace({1:'Yes', 2: 'No'})

In [19]:
# bene_train.dtypes

In [18]:
# calculating Chronic disease count based on all the chronic disease columns
for i in range(bene_test.shape[0]):
    chronicDCnt = 0
    for disease in diseases:
        if bene_test.loc[i,disease] == 'Yes':
                chronicDCnt+=1
        else:
                continue
    bene_test.loc[i,'ChronicDisease_Count'] = chronicDCnt

In [19]:
bene_test.head(2)

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,1943-01-01,2009-12-01,1,1,0,39,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11007,1940-09-01,2009-12-01,1,2,0,45,610,12,12,Yes,Yes,No,No,No,No,Yes,No,Yes,Yes,No,0,0,1490,160,1940,69.0,0,5.0


In [21]:
bene_test.shape

(63968, 29)

In [22]:
bene_test['Gender'].value_counts()

Gender
2    36721
1    27247
Name: count, dtype: int64

In [23]:
# Replacing gender with category
bene_test['Gender'] = bene_test['Gender'].replace({1:'Male', 2:'Female'})

In [24]:
# Replacing race with actual category based on common CMS (Centers for Medicare & Medicaid Services)
bene_test['Race'] = bene_test['Race'].replace({1:'White', 2:'Black or African American',3:'Other', 5:'Hispanic'})

In [25]:
bene_test['Race'].value_counts()

Race
White                        54105
Black or African American     6384
Other                         2147
Hispanic                      1332
Name: count, dtype: int64

In [26]:
# State mapping based on code 

state_mapping = {
    1: 'Alabama', 2: 'Alaska', 3: 'Arizona', 4: 'Arkansas', 5: 'California', 
    6: 'Colorado', 7: 'Connecticut', 8: 'Delaware', 9: 'District of Columbia', 
    10: 'Florida', 11: 'Georgia', 12: 'Hawaii', 13: 'Idaho', 14: 'Illinois', 
    15: 'Indiana', 16: 'Iowa', 17: 'Kansas', 18: 'Kentucky', 19: 'Louisiana', 
    20: 'Maine', 21: 'Maryland', 22: 'Massachusetts', 23: 'Michigan', 
    24: 'Minnesota', 25: 'Mississippi', 26: 'Missouri', 27: 'Montana', 
    28: 'Nebraska', 29: 'Nevada', 30: 'New Hampshire', 31: 'New Jersey', 
    32: 'New Mexico', 33: 'New York', 34: 'North Carolina', 35: 'North Dakota', 
    36: 'Ohio', 37: 'Oklahoma', 38: 'Oregon', 39: 'Pennsylvania', 
    40: 'Rhode Island', 41: 'South Carolina', 42: 'South Dakota', 
    43: 'Tennessee', 44: 'Texas', 45: 'Utah', 46: 'Vermont', 47: 'Virginia', 
    48: 'Washington', 49: 'West Virginia', 50: 'Wisconsin', 51: 'Wyoming',
    52: 'American Samoa', 53: 'Guam', 54: 'Puerto Rico'
}
 
# Replace integer codes with state names
bene_test['State'] = bene_test['State'].replace(state_mapping)

In [26]:
# bene_test['State'].value_counts()

### Inpatient data & Outpatient Data

In [27]:
outp_test.shape, inp_test.shape

((125841, 27), (9551, 30))

In [28]:
outp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11001,CLM392397,2009-06-02,2009-06-02,PRV55962,30,PHY347633,NaN,PHY347633,V5832,V5861,2724,3182,V5869,42731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,BENE11001,CLM430760,2009-06-23,2009-06-23,PRV56112,30,PHY381777,NaN,PHY381777,9594,E9174,4019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,BENE11007,CLM233081,2009-03-07,2009-03-07,PRV56979,200,PHY425311,NaN,PHY425311,7248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11007,CLM496381,2009-07-29,2009-07-29,PRV56573,10,PHY393253,PHY347995,NaN,58889,2449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5939
4,BENE11007,CLM521391,2009-08-12,2009-08-12,PRV56573,10,PHY417685,NaN,PHY382041,V666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [29]:
inp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,NaN,2009-09-09,5789,1068.0,2009-09-16,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,NaN,NaN,NaN,NaN
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,NaN,2008-12-25,5939,1068.0,2009-01-08,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,NaN,5551.0,NaN,NaN,NaN,NaN,NaN
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,NaN,NaN,2009-12-09,4019,1068.0,2009-12-13,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,NaN,2009-06-23,8208,1068.0,2009-07-06,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,NaN,NaN,NaN,NaN
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,NaN,NaN,2009-01-20,4279,1068.0,2009-01-31,880,29654,V142,78702,30503,V140,V4582,V6109,7242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
inp_test['ClaimStartDt'].min()

'2008-11-27'

In [31]:
for col in ['ClaimEndDt','ClaimStartDt', 'DischargeDt', 'AdmissionDt']:  
    inp_test[col] = pd.to_datetime(inp_test[col])

for col in ['ClaimEndDt','ClaimStartDt']:  
    outp_test[col] = pd.to_datetime(outp_test[col])

In [32]:
inp_test['ClaimPeriod'] =  round(((inp_test.ClaimEndDt - inp_test.ClaimStartDt).dt.days),0)
outp_test['ClaimPeriod'] =  round(((outp_test.ClaimEndDt - outp_test.ClaimStartDt).dt.days),0)

In [33]:
inp_test['TimeInHptal'] =  round(((inp_test.DischargeDt - inp_test.AdmissionDt).dt.days),0)

In [34]:
# dropping date columns as no longer needed
inp_test.drop(['DischargeDt','AdmissionDt'],axis=1,inplace=True)

In [35]:
# inp_train['ClmDiagnosisCode_2'].value_counts()

In [36]:
ClmProcedureCode = ['ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',
        'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6',]

ClmDiagnosisCode = ['ClmDiagnosisCode_1',
       'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
       'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7',
       'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

In [37]:
for code in ClmDiagnosisCode:
    inp_test.fillna('-1',inplace=True)
    outp_test.fillna('-1',inplace=True)

def CountingIndex(df,codeList,indexName):
    
    for i in range(df.shape[0]):#
        DiagnosisCnt = 0
        for code in codeList:
            if df.loc[i,code]== '-1' :
                continue  
            else:
                 DiagnosisCnt+=1
        df.loc[i,indexName] = DiagnosisCnt

In [38]:
CountingIndex(inp_test,ClmDiagnosisCode,'Diagnosis Count')
CountingIndex(outp_test,ClmDiagnosisCode,'Diagnosis Count')

In [39]:
CountingIndex(inp_test,ClmProcedureCode,'Procedures Count')
CountingIndex(outp_test,ClmProcedureCode,'Procedures Count')

In [40]:
inp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0


We create a new feature: SameApplicant, it is 1 if the attending physician and the operating physician in this patient data are the same person and 0 if not

In [41]:
def isSamePhysician(df):
    for i in range(0,df.shape[0]):

        if  df.loc[i,['AttendingPhysician']][0] == df.loc[i,['OperatingPhysician']][0]:
            df.loc[i,'SamePhysician'] = 1
        else:
            df.loc[i,'SamePhysician'] = 0

In [42]:
inp_test.OtherPhysician.fillna(0,inplace=True)
outp_test.OtherPhysician.fillna(0,inplace=True)

In [43]:
# outp_train.isna().sum()

In [43]:
isSamePhysician(inp_test)
isSamePhysician(outp_test)

In [44]:
inp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0,0.0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0,0.0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0,0.0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0,0.0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0,0.0


In [45]:
inp_test.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid',
       'DiagnosisGroupCode', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician'],
      dtype='object')

In [46]:
bene_test.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,1943-01-01,2009-12-01,Male,White,0,Pennsylvania,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11007,1940-09-01,2009-12-01,Male,Black or African American,0,Utah,610,12,12,Yes,Yes,No,No,No,No,Yes,No,Yes,Yes,No,0,0,1490,160,1940,69.0,0,5.0
2,BENE11010,1936-07-01,2009-12-01,Female,White,0,South Carolina,30,12,12,No,Yes,No,Yes,Yes,No,Yes,Yes,Yes,No,No,0,0,1170,660,1936,73.0,0,6.0
3,BENE11011,1914-03-01,2009-12-01,Female,Black or African American,0,Alabama,360,12,12,No,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,5000,1068,250,320,1914,96.0,0,6.0
4,BENE11014,1938-04-01,2009-12-01,Female,White,1,Utah,780,12,12,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No,No,21260,2136,120,100,1938,72.0,0,5.0


In [125]:
# bene_train.to_csv('../data/interim/clean2/bene_train.csv')
# bene_train.shape

(138556, 29)

In [48]:
# inp_train.to_csv('../data/interim/clean1/inpatient_train.csv')
# inp_train.shape

(40474, 33)

In [49]:
# outp_train.to_csv('../data/interim/clean1/outpatient_train.csv')
# outp_train.shape

(517737, 31)

In [47]:
inp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician']

In [128]:
# inp_train[inp_imp_cols].to_csv('../data/interim/clean2/inpatient_train.csv')

In [48]:
outp_test.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode', 'ClaimPeriod', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician'],
      dtype='object')

In [49]:
outp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician']

In [129]:
# outp_train[outp_imp_cols].to_csv('../data/interim/clean2/outpatient_train.csv')

### Merging all the files

In [50]:
label_test.head()

,Provider
0,PRV51002
1,PRV51006
2,PRV51009
3,PRV51010
4,PRV51018


In [51]:
label_test.shape

(1353, 1)

In [52]:
label_test['Provider'].nunique()

1259

In [55]:
outp_test['Provider'].nunique()

1259

In [57]:
# label_test['PotentialFraud'].replace('No',0,inplace=True)
# label_test['PotentialFraud'].replace('Yes',1,inplace=True)

# fraud = label_test[label_test['PotentialFraud']==1]
# fraud.shape[0]

In [58]:
inp_test['OPD_Flag'] = 0

In [59]:
inp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician', 'OPD_Flag']

In [60]:
inp_imp_cols

['BeneID',
 'ClaimID',
 'Provider',
 'InscClaimAmtReimbursed',
 'DeductibleAmtPaid',
 'ClaimPeriod',
 'TimeInHptal',
 'Diagnosis Count',
 'Procedures Count',
 'SamePhysician',
 'OPD_Flag']

In [61]:
outp_test['TimeInHptal'] = 0
outp_test['OPD_Flag'] = 1

In [62]:
outp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal','Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag']

In [63]:
outp_imp_cols

['BeneID',
 'ClaimID',
 'Provider',
 'InscClaimAmtReimbursed',
 'DeductibleAmtPaid',
 'ClaimPeriod',
 'TimeInHptal',
 'Diagnosis Count',
 'Procedures Count',
 'SamePhysician',
 'OPD_Flag']

In [64]:
outp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode,ClaimPeriod,Diagnosis Count,Procedures Count,SamePhysician,TimeInHptal,OPD_Flag
0,BENE11001,CLM392397,2009-06-02,2009-06-02,PRV55962,30,PHY347633,-1,PHY347633,V5832,V5861,2724,3182,V5869,42731,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,6.0,0.0,0.0,0,1
1,BENE11001,CLM430760,2009-06-23,2009-06-23,PRV56112,30,PHY381777,-1,PHY381777,9594,E9174,4019,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,3.0,0.0,0.0,0,1
2,BENE11007,CLM233081,2009-03-07,2009-03-07,PRV56979,200,PHY425311,-1,PHY425311,7248,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,1.0,0.0,0.0,0,1
3,BENE11007,CLM496381,2009-07-29,2009-07-29,PRV56573,10,PHY393253,PHY347995,-1,58889,2449,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,5939,0,2.0,0.0,0.0,0,1
4,BENE11007,CLM521391,2009-08-12,2009-08-12,PRV56573,10,PHY417685,-1,PHY382041,V666,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,1.0,0.0,0.0,0,1


In [65]:
inp_test.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0,0.0,0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0,0.0,0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0,0.0,0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0,0.0,0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0,0.0,0


In [66]:
inp_test.shape, outp_test.shape

((9551, 34), (125841, 33))

In [67]:
ip_op_test = pd.concat([inp_test[inp_imp_cols], outp_test[outp_imp_cols]])
ip_op_test.shape

(135392, 11)

In [68]:
ip_op_test.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag
0,BENE11014,CLM67387,PRV57070,9000,1068.0,7,7,10.0,2.0,0.0,0
1,BENE11017,CLM31237,PRV54750,14000,1068.0,14,14,9.0,1.0,0.0,0
2,BENE11026,CLM78930,PRV53758,2000,1068.0,4,4,9.0,0.0,0.0,0
3,BENE11031,CLM56810,PRV55825,16000,1068.0,13,13,10.0,2.0,0.0,0
4,BENE11085,CLM34625,PRV52338,19000,1068.0,11,11,8.0,0.0,0.0,0


In [114]:
# ip_op_train.to_clipboard()

In [118]:
# ip_op_train = pd.merge(left=inp_train[inp_imp_cols], right=outp_train[outp_imp_cols], 
#                 how='outer', left_on='BeneID', right_on='BeneID')
# ip_op_train.shape

In [69]:
ip_op_test.columns

Index(['BeneID', 'ClaimID', 'Provider', 'InscClaimAmtReimbursed',
       'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag'],
      dtype='object')

In [70]:
# Joining the IP_OP dataset with the BENE data
test_df = pd.merge(left=ip_op_test, right=bene_test, left_on='BeneID', right_on='BeneID',how='inner')
test_df.shape

(135392, 39)

In [121]:
test_df.to_clipboard()

In [71]:
test_df.to_csv('../../data/validation/testing_data.csv')

In [72]:
test_df.isna().sum()

BeneID                             0
ClaimID                            0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
ClaimPeriod                        0
TimeInHptal                        0
Diagnosis Count                    0
Procedures Count                   0
SamePhysician                      0
OPD_Flag                           0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
C